In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py

In [ ]:
#passband extinction coefficients - multiply the "E" by these to get extinction in magnitudes -
g_co = 3.518
r_co = 2.617
i_co = 1.971
z_co = 1.549
y_co = 1.263
j_co = 0.7927
h_co = 0.4690
ks_co = 0.3026
#presumably the order for mag follows this?

#these coefficients are given in the green et al 2019 paper

# making a mega file of the data from green et al in the directions of ell ~ 180 and ell ~ 90. 
The data come were downloaded from here: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/AV9GXO

There are about a hundered files in this directory, and the way that they're ordered doesn't make any sense. Consecutive files don't necessarily correspond to patches on the sky that are near one another, and some files have really random pieces of the sky together. SO - this means that I unfortunately have to loop through all of these files to build my mega file of stars at the latitudes, longitudes that I want

In [ ]:
import glob
dirlist = sorted(glob.glob("/datascope/subaru/data/cmdfit/dustmaps/green/stellar_params*.h5"))

ls = np.array([])
bs = np.array([])
median_extinction = np.array([])
median_dm = np.array([])
median_Mr = np.array([])
g_mag = np.array([])
r_mag = np.array([])
i_mag = np.array([])
z_mag = np.array([])
y_mag = np.array([])
j_mag = np.array([])
h_mag = np.array([])
ks_mag = np.array([])
g_mag_err = np.array([])
r_mag_err = np.array([])
i_mag_err = np.array([])
z_mag_err = np.array([])
y_mag_err = np.array([])
j_mag_err = np.array([])
h_mag_err = np.array([])
ks_mag_err = np.array([])
plx = np.array([])
plx_err = np.array([])
gaia_id = np.array([])
gaia_ag = np.array([])
#gaia_ag_low = np.array([])
#gaia_ag_high = np.array([])
gaia_ebr = np.array([])
#gaia_ebr_low = np.array([])
#gaia_ebr_high = np.array([])
median_feh = np.array([])
chisq_cut = np.array([])
idd = np.array([])
#'S16' was 'O'
mega_file_dtype = [('idd','S20'), ('l','f'), ('b','f'),('plx','f'), ('plx_err','f'), ('g_mag','f'), 
                   ('g_mag_err','f'), ('r_mag','f'), ('r_mag_err','f'), ('i_mag','f'), ('i_mag_err','f'),
                   ('z_mag','f'),('z_mag_err','f'), ('y_mag','f'), ('y_mag_err','f'), ('j_mag','f'), 
                   ('j_mag_err','f'), ('h_mag','f'), ('h_mag_err','f'), ('ks_mag','f'), ('ks_mag_err','f'), 
                   ('gaia_id', 'S16'),('gaia_a_g','f'), ('gaia_ebr','f'), ('median_dm', 'f'), 
                   ('median_extinction','f'),
                   ('median_Mr','f'), ('median_feh','f'), ('chisq', 'f'), ('dered_g_mag','f'),
                  ('dered_r_mag','f'),('dered_i_mag','f'),('dered_z_mag','f'), ('dered_y_mag','f'),
                  ('dered_j_mag','f'), ('dered_h_mag','f'), ('dered_ks_mag','f')]
total_len = 0



for file in dirlist:
    print(file)
    f = h5py.File(file, "r")
    # List all groups
    chisq_group_key = list(f.keys())[0]
    data_group_key = list(f.keys())[1]
    gaia_group_key = list(f.keys())[2]
    metadata_group_key = list(f.keys())[3]
    percentiles_group_key = list(f.keys())[4]
    samples_group_key = list(f.keys())[5]
    items = list(f[data_group_key])

    chisq = f[chisq_group_key]
    # Get the data
    data = f[data_group_key]
    #gaia
    gaia = f[gaia_group_key]
    meta = f[metadata_group_key]
    percentile = f[percentiles_group_key]
    samples = f[samples_group_key]
    
    for i in items:
        idx = np.where(((170<= meta[i]['l']) & (meta[i]['l']<= 190)) | ((80 <= meta[i]['l']) & (meta[i]['l']<= 100)))
        #np.where(((170>= meta[i]['l']) & (meta[i]['l']<= 190)) | ((80 >= meta[i]['l']) & (meta[i]['l']<= 100)))
        ls = np.append(ls, meta[i]['l'][idx])
        bs = np.append(bs, meta[i]['b'][idx])

        total_len = ls.shape[0]

        idd = np.append(idd, meta[i]['obj_id'][idx].astype('str'))
        chisq_cut = np.append(chisq_cut, chisq[i][idx])

        median_extinction = np.append(median_extinction, percentile[i]['E'][:,1][idx])
        median_dm = np.append(median_dm, percentile[i]['dm'][:,1][idx])
        median_Mr = np.append(median_Mr, percentile[i]['Mr'][:,1][idx])
        median_feh = np.append(median_feh, percentile[i]['FeH'][:,1][idx])



        g_mag = np.append(g_mag, data[i]['mag'][:,0][idx])    
        r_mag = np.append(r_mag, data[i]['mag'][:,1][idx])    
        i_mag = np.append(i_mag, data[i]['mag'][:,2][idx])    
        z_mag = np.append(z_mag, data[i]['mag'][:,3][idx])    
        y_mag = np.append(y_mag, data[i]['mag'][:,4][idx])    
        j_mag = np.append(j_mag, data[i]['mag'][:,5][idx])    
        h_mag = np.append(h_mag, data[i]['mag'][:,6][idx])    
        ks_mag = np.append(ks_mag, data[i]['mag'][:,7][idx])

        g_mag_err = np.append(g_mag_err, data[i]['mag_err'][:,0][idx])    
        r_mag_err = np.append(r_mag_err, data[i]['mag_err'][:,1][idx])    
        i_mag_err = np.append(i_mag_err, data[i]['mag_err'][:,2][idx])    
        z_mag_err = np.append(z_mag_err, data[i]['mag_err'][:,3][idx])    
        y_mag_err = np.append(y_mag_err, data[i]['mag_err'][:,4][idx])    
        j_mag_err = np.append(j_mag_err, data[i]['mag_err'][:,5][idx])    
        h_mag_err = np.append(h_mag_err, data[i]['mag_err'][:,6][idx])    
        ks_mag_err = np.append(ks_mag_err, data[i]['mag_err'][:,7][idx])

        plx = np.append(plx, data[i]['plx'][idx])
        plx_err = np.append(plx_err, data[i]['plx_err'][idx])
        gaia_id = np.append(gaia_id, gaia[i]['gaia_id'].astype('str')[idx])
        gaia_ag = np.append(gaia_ag, gaia[i]['a_g'][idx])
        gaia_ebr = np.append(gaia_ebr, gaia[i]['ebr'][idx])

    print('closing')
    f.close()

print('on to dered')
dered_g_mag = g_mag - g_co*median_extinction
dered_r_mag = r_mag - r_co*median_extinction
dered_i_mag = i_mag - i_co*median_extinction
dered_z_mag = z_mag - z_co*median_extinction
dered_y_mag = y_mag - y_co*median_extinction

dered_j_mag = j_mag - j_co*median_extinction
dered_h_mag = h_mag - h_co*median_extinction
dered_ks_mag = ks_mag - ks_co*median_extinction

print(total_len, idd.shape)

savearray = np.empty(total_len,dtype=mega_file_dtype)

savearray['idd'] = idd
savearray['l'] = ls
savearray['b'] = bs
savearray['median_extinction'] = median_extinction
savearray['median_dm'] = median_dm 
savearray['median_Mr'] = median_Mr
savearray['g_mag'] = g_mag
savearray['r_mag'] = r_mag
savearray['i_mag'] = i_mag
savearray['z_mag'] = z_mag
savearray['y_mag'] = y_mag
savearray['j_mag'] = j_mag
savearray['h_mag'] = h_mag
savearray['ks_mag'] = ks_mag
savearray['g_mag_err'] = g_mag_err
savearray['r_mag_err'] = r_mag_err
savearray['i_mag_err'] = i_mag_err
savearray['z_mag_err'] = z_mag_err
savearray['y_mag_err'] = y_mag_err
savearray['j_mag_err'] = j_mag_err
savearray['h_mag_err'] = h_mag_err
savearray['ks_mag_err'] = ks_mag_err
savearray['plx'] = plx
savearray['plx_err'] = plx_err
savearray['gaia_id'] = gaia_id
savearray['gaia_a_g'] = gaia_ag
savearray['gaia_ebr'] = gaia_ebr
savearray['median_feh'] = median_feh
savearray['chisq'] = chisq_cut
savearray['dered_g_mag'] = dered_g_mag
savearray['dered_r_mag'] = dered_r_mag
savearray['dered_i_mag'] = dered_i_mag
savearray['dered_z_mag'] = dered_z_mag
savearray['dered_y_mag'] = dered_y_mag
savearray['dered_j_mag'] = dered_j_mag
savearray['dered_h_mag'] = dered_h_mag
savearray['dered_ks_mag'] = dered_ks_mag

print('saving')


hdf5 = h5py.File('combined_green_files_l180_l90.h5', 'w')
#h5py.File('combined_green_files_l180_l90.h5', 'w')
#savearray = np.empty(total_len,dtype=mega_file_dtype)
ds = hdf5.create_dataset('test dataset', data = savearray, #(total_len,),
                              dtype=mega_file_dtype)    
hdf5.close()

In [ ]:
#check this beautiful file we just made -
hdf5 = h5py.File('combined_green_files_l180_l90.h5', 'w')
#savearray = np.empty(total_len,dtype=mega_file_dtype)
ds = hdf5.create_dataset('test dataset', data = savearray, #(total_len,),
                              dtype=mega_file_dtype)    
hdf5.close()

# -----

# below is just testing stuff out/figuring out what should go in the final code that is above

In [ ]:
import glob
dirlist = sorted(glob.glob("/datascope/subaru/data/cmdfit/dustmaps/green/stellar_params*.h5"))[10:15]

ls = np.array([])
bs = np.array([])
median_extinction = np.array([])
median_dm = np.array([])
median_Mr = np.array([])
g_mag = np.array([])
r_mag = np.array([])
i_mag = np.array([])
z_mag = np.array([])
y_mag = np.array([])
j_mag = np.array([])
h_mag = np.array([])
ks_mag = np.array([])
g_mag_err = np.array([])
r_mag_err = np.array([])
i_mag_err = np.array([])
z_mag_err = np.array([])
y_mag_err = np.array([])
j_mag_err = np.array([])
h_mag_err = np.array([])
ks_mag_err = np.array([])
plx = np.array([])
plx_err = np.array([])
gaia_id = np.array([])
gaia_ag = np.array([])
#gaia_ag_low = np.array([])
#gaia_ag_high = np.array([])
gaia_ebr = np.array([])
#gaia_ebr_low = np.array([])
#gaia_ebr_high = np.array([])
median_feh = np.array([])
chisq_cut = np.array([])
idd = np.array([])
#'S16' was 'O'
mega_file_dtype = [('idd','S18'), ('l','f'), ('b','f'),('plx','f'), ('plx_err','f'), ('g_mag','f'), 
                   ('g_mag_err','f'), ('r_mag','f'), ('r_mag_err','f'), ('i_mag','f'), ('i_mag_err','f'),
                   ('z_mag','f'),('z_mag_err','f'), ('y_mag','f'), ('y_mag_err','f'), ('j_mag','f'), 
                   ('j_mag_err','f'), ('h_mag','f'), ('h_mag_err','f'), ('ks_mag','f'), ('ks_mag_err','f'), 
                   ('gaia_id', 'S16'),('gaia_a_g','f'), ('gaia_ebr','f'), ('median_dm', 'f'), 
                   ('median_extinction','f'),
                   ('median_Mr','f'), ('median_feh','f'), ('chisq', 'f'), ('dered_g_mag','f'),
                  ('dered_r_mag','f'),('dered_i_mag','f'),('dered_z_mag','f'), ('dered_y_mag','f'),
                  ('dered_j_mag','f'), ('dered_h_mag','f'), ('dered_ks_mag','f')]
total_len = 0



for file in dirlist:
    print(file)
    f = h5py.File(file, "r")
    # List all groups
    chisq_group_key = list(f.keys())[0]
    data_group_key = list(f.keys())[1]
    gaia_group_key = list(f.keys())[2]
    metadata_group_key = list(f.keys())[3]
    percentiles_group_key = list(f.keys())[4]
    samples_group_key = list(f.keys())[5]
    items = list(f[data_group_key])

    chisq = f[chisq_group_key]
    # Get the data
    data = f[data_group_key]
    #gaia
    gaia = f[gaia_group_key]
    meta = f[metadata_group_key]
    percentile = f[percentiles_group_key]
    samples = f[samples_group_key]
    for i in items:
        
        ls = np.append(ls, meta[i]['l'])
        bs = np.append(bs, meta[i]['b'])
        
        total_len = ls.shape[0]
        
        idd = np.append(idd, meta[i]['obj_id'].astype('str'))
        chisq_cut = np.append(chisq_cut, chisq[i])
        
        median_extinction = np.append(median_extinction, percentile[i]['E'][:,1])
        median_dm = np.append(median_dm, percentile[i]['dm'][:,1])
        median_Mr = np.append(median_Mr, percentile[i]['Mr'][:,1])
        median_feh = np.append(median_feh, percentile[i]['FeH'][:,1])



        g_mag = np.append(g_mag, data[i]['mag'][:,0])    
        r_mag = np.append(r_mag, data[i]['mag'][:,1])    
        i_mag = np.append(i_mag, data[i]['mag'][:,2])    
        z_mag = np.append(z_mag, data[i]['mag'][:,3])    
        y_mag = np.append(y_mag, data[i]['mag'][:,4])    
        j_mag = np.append(j_mag, data[i]['mag'][:,5])    
        h_mag = np.append(h_mag, data[i]['mag'][:,6])    
        ks_mag = np.append(ks_mag, data[i]['mag'][:,7])

        g_mag_err = np.append(g_mag_err, data[i]['mag_err'][:,0])    
        r_mag_err = np.append(r_mag_err, data[i]['mag_err'][:,1])    
        i_mag_err = np.append(i_mag_err, data[i]['mag_err'][:,2])    
        z_mag_err = np.append(z_mag_err, data[i]['mag_err'][:,3])    
        y_mag_err = np.append(y_mag_err, data[i]['mag_err'][:,4])    
        j_mag_err = np.append(j_mag_err, data[i]['mag_err'][:,5])    
        h_mag_err = np.append(h_mag_err, data[i]['mag_err'][:,6])    
        ks_mag_err = np.append(ks_mag_err, data[i]['mag_err'][:,7])
        
        plx = np.append(plx, data[i]['plx'])
        plx_err = np.append(plx_err, data[i]['plx_err'])
        #gaia_id = np.append(gaia_id, gaia[i]['gaia_id'].astype('str'))
        #gaia_ag = np.append(gaia_ag, gaia[i]['a_g'])
        #gaia_ag_low = np.append(gaia_ag_low, gaia[i]['a_g_low'])
        #gaia_ag_high = np.append(gaia_ag_high, gaia[i]['a_g_high'])
        #gaia_ebr = np.append(gaia_ebr, gaia[i]['ebr'])
        #gaia_ebr_low = np.append(gaia_ebr_low, gaia[i]['ebr_low'])
        #gaia_ebr_high = np.append(gaia_ebr_hgih, gaia[i]['ebr_high'])

        
    #np.save('combined_green_files', savearray)
    


#np.save('combined_green_files', savearray)
#columns in chisq float32
#columns in data [('plx', '<f4'), ('plx_err', '<f4'), ('mag', '<f4', (8,)), ('mag_err', '<f4', (8,))]
#columns in gaia [('gaia_id', '<u8'), ('a_g', '<f4'), ('a_g_low', '<f4'), ('a_g_high', '<f4'), 
                #('ebr', '<f4'), ('ebr_low', '<f4'), ('ebr_high', '<f4')]
#columns in meta [('obj_id', '<u8'), ('l', '<f8'), ('b', '<f8')]
#columns in percentile [('dm', '<f4'), ('E', '<f4'), ('Mr', '<f4'), ('FeH', '<f4')]
#columns in samples [('dm', '<f4'), ('E', '<f4'), ('Mr', '<f4'), ('FeH', '<f4'), 
                    #('ln_likelihood', '<f4'), ('ln_prior', '<f4')]

In [ ]:
#

In [ ]:
dered_g_mag = g_mag - g_co*median_extinction
dered_r_mag = r_mag - r_co*median_extinction
dered_i_mag = i_mag - i_co*median_extinction
dered_z_mag = z_mag - z_co*median_extinction
dered_y_mag = y_mag - y_co*median_extinction

dered_j_mag = j_mag - j_co*median_extinction
dered_h_mag = h_mag - h_co*median_extinction
dered_ks_mag = ks_mag - ks_co*median_extinction

print(total_len, idd.shape)


#'S16' was 'O'
mega_file_dtype = [('idd','S18'), ('l','f'), ('b','f'),('plx','f'), ('plx_err','f'), ('g_mag','f'), 
                   ('g_mag_err','f'), ('r_mag','f'), ('r_mag_err','f'), ('i_mag','f'), ('i_mag_err','f'),
                   ('z_mag','f'),('z_mag_err','f'), ('y_mag','f'), ('y_mag_err','f'), ('j_mag','f'), 
                   ('j_mag_err','f'), ('h_mag','f'), ('h_mag_err','f'), ('ks_mag','f'), ('ks_mag_err','f'), 
                  # ('gaia_id', 'S16'),('gaia_a_g','f'), ('gaia_ebr','f'), 
                   ('median_dm', 'f'), ('median_extinction','f'),
                   ('median_Mr','f'), ('median_feh','f'), ('chisq', 'f'), ('dered_g_mag','f'),
                  ('dered_r_mag','f'),('dered_i_mag','f'),('dered_z_mag','f'), ('dered_y_mag','f'),
                  ('dered_j_mag','f'), ('dered_h_mag','f'), ('dered_ks_mag','f')]

savearray = np.empty(total_len,dtype=mega_file_dtype)

savearray['idd'] = idd
savearray['l'] = ls
savearray['b'] = bs
savearray['median_extinction'] = median_extinction
savearray['median_dm'] = median_dm 
savearray['median_Mr'] = median_Mr
savearray['g_mag'] = g_mag
savearray['r_mag'] = r_mag
savearray['i_mag'] = i_mag
savearray['z_mag'] = z_mag
savearray['y_mag'] = y_mag
savearray['j_mag'] = j_mag
savearray['h_mag'] = h_mag
savearray['ks_mag'] = ks_mag
savearray['g_mag_err'] = g_mag_err
savearray['r_mag_err'] = r_mag_err
savearray['i_mag_err'] = i_mag_err
savearray['z_mag_err'] = z_mag_err
savearray['y_mag_err'] = y_mag_err
savearray['j_mag_err'] = j_mag_err
savearray['h_mag_err'] = h_mag_err
savearray['ks_mag_err'] = ks_mag_err
savearray['plx'] = plx
#savearray['plx_err'] = plx_err
#savearray['gaia_id'] = gaia_id
#savearray['gaia_a_g'] = gaia_ag
#savearray['gaia_ebr'] = gaia_ebr
savearray['median_feh'] = median_feh
savearray['chisq'] = chisq_cut
savearray['dered_g_mag'] = dered_g_mag
savearray['dered_r_mag'] = dered_r_mag
savearray['dered_i_mag'] = dered_i_mag
savearray['dered_z_mag'] = dered_z_mag
savearray['dered_y_mag'] = dered_y_mag
savearray['dered_j_mag'] = dered_j_mag
savearray['dered_h_mag'] = dered_h_mag
savearray['dered_ks_mag'] = dered_ks_mag

print('saving')

In [ ]:
#hdf5 = h5py.File('combined_green_files_0_5.h5', 'w')
#hdf5 = h5py.File('combined_green_files_5_10.h5', 'w')
hdf5 = h5py.File('combined_green_files_10_15.h5', 'w')
#savearray = np.empty(total_len,dtype=mega_file_dtype)
ds = hdf5.create_dataset('test dataset', data = savearray, #(total_len,),
                              dtype=mega_file_dtype)    
hdf5.close()

In [ ]:
print('done')
hdf5.close()
len(mega_file_dtype)

In [ ]:
np.save('combined_green_files_backup', savearray)

In [ ]:
idd.shape[0],savearray.shape

In [ ]:
np.random.uniform(-2,2)

In [ ]:
np.load('combined_green_files.npy',allow_pickle=True)